In [1]:
import numpy
import pandas

import conntility

from scipy.spatial.distance import cdist
import tqdm

from matplotlib import pyplot as plt

## Part 1:
Measure the strengths of statistical interactions in connectivity based on individual morphologies that are not capture by a fourth-order simplifed model

In [2]:
fn_rat = "/gpfs/bbp.cscs.ch/project/proj159/home/barros/conn_matrix/Rat_623um_squared_struc_conmat_filtered_compressed.h5"
fn_human = "/gpfs/bbp.cscs.ch/project/proj159/home/barros/conn_matrix/Human_960um_squared_struc_conmat_filtered_compressed.h5"
fn_human_func = "/gpfs/bbp.cscs.ch/project/proj159/home/barros/conn_matrix/Human_NEWfunct_conmat_filtered_compressed.h5"

# https://doi.org/10.5281/zenodo.13849415
fn_microns = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/microns/microns_mm3_connectome_v1181.h5"
fn_control = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/third_order_mdl_instances_human.h5"

loaded = "human_functional"  # Set to human to analyze that instead

if loaded == "rat":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_rat)
    col_y = "depth"
    col_xz = ["ss_flat_x", "ss_flat_y"]
    bin_sz = 50.0
elif loaded == "human":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_human)
    col_y = "y"
    col_xz = ["x", "z"]
    bin_sz = 50.0 * 1.76
elif loaded == "human_functional":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_human_func)
    col_y = "y"
    col_xz = ["x", "z"]
    bin_sz = 50.0 * 1.76
elif loaded == "microns":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_microns, "condensed")
    M_h = M_h.index("cell_type").isin(["L2a", "L2b", "L2c", "L3a", "L3b"])
    for col in ["x", "y", "z"]:
        M_h.add_vertex_property(col, M_h.vertices[col + "_nm"].values / 1000)
    M_h.add_vertex_property("mtype", M_h.vertices["cell_type"].values)
    col_y = "y"
    col_xz = ["x", "z"]
    bin_sz = 50.0
elif loaded == "control":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_control, "instance1")
    col_y = "y"
    col_xz = ["x", "z"]
    bin_sz = 50.0 * 1.76
    
def make_spatial_bins(M_h, cols, bin_sz):
    _data = M_h.vertices[cols]
    delta = _data.max() - _data.min()

    sz = numpy.sqrt((delta.values ** 2).sum())
    if len(delta) == 1: # case 1d: negative and positive bins
        bins = numpy.arange(0, (bin_sz * numpy.ceil(sz / bin_sz)) + bin_sz, bin_sz)
        bins = numpy.hstack([-bins[:0:-1], bins])
    else: # case 2d: Only positive bins, but exclude 0 dist
        bins = numpy.arange(0, (bin_sz * numpy.ceil(sz / bin_sz)) + bin_sz, bin_sz)
        bins = numpy.hstack([[0, 1E-12], bins[1:]])
    return bins

dbins_xz = make_spatial_bins(M_h, col_xz, bin_sz)
binid_xz = numpy.arange(0, len(dbins_xz) + 1)

dbins_y = make_spatial_bins(M_h, [col_y], bin_sz)
binid_y = numpy.arange(0, len(dbins_y) + 1)

bin_centers_y = 0.5 * (dbins_y[:-1] + dbins_y[1:])
bin_centers_xz = 0.5 * (dbins_xz[1:-1] + dbins_xz[2:])

M_h.vertices["mtype"].value_counts()

mtype
L2_PTPC    9206
L3_PTPC    6440
L3_STPC    3950
L3_PC       468
IN            0
BC            0
L4_PC         0
L5_PC         0
L6_PC         0
Name: count, dtype: int64

In [3]:
from scipy.spatial import KDTree

_coords = col_xz + [col_y]
tree = KDTree(M_h.vertices[_coords].values)

_, nn_id = tree.query(M_h.vertices[_coords], k=2)
nn_id = nn_id[:, 1]  # nn_id[:, 0] is the original node, which has distance 0. nn_id[:, 1] is neighbor

# Lookup from pre / post ids to edge ids
edge_id_lookup = M_h._edge_indices.reset_index(drop=True).reset_index().set_index(["row", "col"])["index"]

For this analysis we build a table as follows:

For each possible xz/y bin and value of n and m:
    We count the number of instances where a neuron N_a at offset xz, y from another neuron N_b has n touches/synapses with N_b, and the nearest neighbor of N_a has m touches/synapses with N_b.

In [4]:
def for_pre_chunk(chunk_pre):
    # Which offset bin the pairs fall into
    Dxz = cdist(M_h.vertices.iloc[chunk_pre][col_xz], M_h.vertices[col_xz]) # PRE X POST
    Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1. That is the one to exclude.

    Dy = M_h.vertices.iloc[chunk_pre][[col_y]].values - M_h.vertices[[col_y]].values.transpose() # PRE X POST
    Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
    
    # Numer of touches between them
    j, i = numpy.meshgrid(numpy.arange(len(M_h)), chunk_pre) # Chunk is i.: Presyn.
    assert j.shape == Dy.shape
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": i.flatten(), "col": j.flatten()}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count[v] = M_h.edges["count"].values[edge_ids[v]]
    
    # Number of touches with nearest neighbor of the presynaptic neuron
    _pre = nn_id[i.flatten()]; _post = j.flatten()
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": _pre, "col": _post}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count_nnpre = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count_nnpre[v] = M_h.edges["count"].values[edge_ids[v]]
    collision_pre = _pre != _post
    
    # Number of touches with nearest neighbor of the postsynaptic neuron
    _pre = i.flatten(); _post = nn_id[j.flatten()]
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": _pre, "col": _post}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count_nnpost = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count_nnpost[v] = M_h.edges["count"].values[edge_ids[v]]
    collision_post = _pre != _post
    
    # Count instances of each
    ret_pre = pandas.DataFrame({
        "xz": Dxz.flatten()[collision_pre],
        "y": Dy.flatten()[collision_pre],
        "touches_pair": touch_count[collision_pre],
        "touches_nn_pre": touch_count_nnpre[collision_pre],
    }).value_counts()
    
    ret_post = pandas.DataFrame({
        "xz": Dxz.flatten()[collision_post],
        "y": Dy.flatten()[collision_post],
        "touches_pair": touch_count[collision_post],
        "touches_nn_post": touch_count_nnpost[collision_post],
    }).value_counts()
    return ret_pre, ret_post
    


In [ ]:
#In each offset-bin: How many pairs exist?
full_master_pre = []
full_master_post = []
chunk_sz = 1000
chunking = numpy.arange(0, len(M_h) + chunk_sz, chunk_sz)

chunk_pre = numpy.arange(chunking[0], numpy.minimum(chunking[1], len(M_h)))
full_master_pre, full_master_post = for_pre_chunk(chunk_pre)

for a, b in tqdm.tqdm(list(zip(chunking[1:-1], chunking[2:]))):
    chunk_pre = numpy.arange(a, numpy.minimum(b, len(M_h)))
    new_master_pre, new_master_post = for_pre_chunk(chunk_pre)
    full_master_pre = full_master_pre.add(new_master_pre, fill_value=0)
    full_master_post = full_master_post.add(new_master_post, fill_value=0)


 75%|███████▌  | 15/20 [03:50<01:17, 15.41s/it]

In [ ]:
full_master_pre = full_master_pre.drop(-1, axis=0).reset_index()
full_master_post = full_master_post.drop(-1, axis=0).reset_index()


assert (full_master_pre[["xz", "y"]] >= 0).all().all()
assert (full_master_post[["xz", "y"]] >= 0).all().all()

assert (full_master_pre["xz"] < len(binid_xz)).all()
assert (full_master_pre["y"] < len(binid_y)).all()
assert (full_master_post["xz"] < len(binid_xz)).all()
assert (full_master_post["y"] < len(binid_y)).all()

In [ ]:
full_master_pre["xz"] = bin_centers_xz[full_master_pre["xz"]]
full_master_pre["y"] = bin_centers_y[full_master_pre["y"]]

full_master_post["xz"] = bin_centers_xz[full_master_post["xz"]]
full_master_post["y"] = bin_centers_y[full_master_post["y"]]

In [ ]:
out_fn = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/connectivity_higher_order_effect_24-11-11.h5"

full_master_pre.to_hdf(out_fn, key="{0}/pre".format(loaded))
full_master_post.to_hdf(out_fn, key="{0}/post".format(loaded))

## Part 2:
The above characterizes statistical interactions within one spatial bin. Here, we consider interactions _between_ bins. 

For this analysis we build a table as follows:

For each possible xz/y bin, value of n neuron N_b:
    We count the number of instances where a neuron N_a at offset xz, y from N_b has n touches/synapses with N_b. The difference  with respect to the previous analysis is that we count separately for each neuron N_b

In [ ]:
edge_id_lookup = M_h._edge_indices.reset_index(drop=True).reset_index().set_index(["row", "col"])["index"]

def for_chunk(chunk, interaction_for="pre"):
    # Which offset bin the pairs fall into
    if interaction_for == "pre": # Pre means: from chunk to all neurons
        Dxz = cdist(M_h.vertices.iloc[chunk][col_xz], M_h.vertices[col_xz]) # PRE X POST
        Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

        Dy = M_h.vertices.iloc[chunk][[col_y]].values - M_h.vertices[[col_y]].values.transpose() # PRE X POST
        Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
        
        j, i = numpy.meshgrid(numpy.arange(len(M_h)), chunk) # chunk is i: presyn. => consider outgoing
        node_id = i.flatten()
    elif interaction_for == "post":
        Dxz = cdist(M_h.vertices[col_xz], M_h.vertices.iloc[chunk][col_xz]) # PRE X POST
        Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

        Dy = M_h.vertices[[col_y]].values - M_h.vertices.iloc[chunk][[col_y]].values.transpose() # PRE X POST
        Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
        
        j, i = numpy.meshgrid(chunk, numpy.arange(len(M_h))) # chunk is j: postsyn. => consider incoming
        node_id = j.flatten()
    
    # Numer of touches between them
    assert j.shape == Dy.shape
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": i.flatten(), "col": j.flatten()}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count[v] = M_h.edges["count"].values[edge_ids[v]]
    touch_count.reshape(Dy.shape)
        
    # Count instances of each
    ret = pandas.DataFrame({
        "xz": Dxz.flatten(),
        "y": Dy.flatten(),
        "touches_pair": touch_count,
        "node_id": node_id,
    }).value_counts()
    ret.name = "count"
    return ret
    


In [ ]:
df_for_pre = []
df_for_post = []

chunk_sz = 2000
chunking = numpy.arange(0, len(M_h) + chunk_sz, chunk_sz)

for a, b in tqdm.tqdm(list(zip(chunking[:-1], chunking[1:]))):
    chunk = numpy.arange(a, numpy.minimum(b, len(M_h)))
    df_for_pre.append(for_chunk(chunk, interaction_for="pre"))
    df_for_post.append(for_chunk(chunk, interaction_for="post"))
    

df_for_pre = pandas.concat(df_for_pre, axis=0).drop(-1, axis=0, level="xz")
df_for_post = pandas.concat(df_for_post, axis=0).drop(-1, axis=0, level="xz")

Next, we convert the information in the table as follows: For each offset bin and neuron calculate the number of connected neurons in that bin and the total number of neurons (connected or not).

In [ ]:
# We introduce a threshold for minimum number of touches to make the results sparser.
# For Microns connectivity or functional connectivity set to 1.
thresh = 1  # HOW MANY TOUCHES NEED TO BE REACHED FOR A CONNECTION

def average_P(df_in):
    return (df_in["N"] * df_in["P"]).sum() / df_in["N"].sum()
    
# For each pre- or post-neuron: How many partners does it have in each offset bin (N) and what fraction is connected (P)?
def count_stats_per_spatial_bin(df_in, add_normalized=True, more_normalized=True):
    def count_n_and_p(df_in):
        M = df_in.pivot(index="touches_pair", columns="node_id", values="count") # "node_id"
        V = pandas.concat([M.sum(axis=0), M.loc[thresh:].sum(axis=0) / M.sum(axis=0)], axis=1, keys=["N", "P"])
        return V
    df_out = df_in.reset_index().groupby(["xz", "y"]).apply(count_n_and_p)
    
    df_out["M"] = df_out["N"] * df_out["P"]
    if add_normalized or more_normalized:
        p_per_nrn = df_out.groupby("node_id").apply(average_P)
    if add_normalized:
        df_out["E"] = df_out["N"] * p_per_nrn
        df_out["dM"] = df_out["M"] - df_out["E"]
        df_out["dP"] = df_out["P"] - df_out["E"] / df_out["N"]
    if more_normalized:
        mean_p_profile = df_out.groupby(["xz", "y"])["P"].mean()
        p_per_nrn = p_per_nrn / p_per_nrn.mean()
        expected_profile = mean_p_profile.values.reshape((-1, 1)) * p_per_nrn.values.reshape((1, -1))
        expected_profile = pandas.DataFrame(expected_profile, index=mean_p_profile.index, columns=p_per_nrn.index)
        expected_profile = expected_profile.stack()
        df_out["E_spat"] = df_out["N"] * expected_profile
        df_out["dM_spat"] = df_out["M"] - df_out["E_spat"]
        df_out["dP_spat"] = df_out["P"] - expected_profile
    return df_out

df_n_p_pre = count_stats_per_spatial_bin(df_for_pre)
df_n_p_post = count_stats_per_spatial_bin(df_for_post)


In [ ]:
from scipy import sparse

def count_number_of_matching_pairs(df_in):
    nn = (~numpy.isnan(df_in)).astype(int)
    nn = sparse.csc_matrix(nn.values)
    nncount = nn.transpose() * nn
    nncount = pandas.DataFrame(numpy.array(nncount.todense()),
                               index=df_in.columns, columns=df_in.columns)
    return nncount

# What is the correlation between observed values of "P" over pre- or post-neurons in a given offset bin?
def observations_to_cc_of_offset_bins(df_in, mode="valid", use="P", thresh_pairs=100, thresh_count=10):
    df_in = df_in[df_in["N"] >= thresh_count]
    df_cc = df_in.reset_index()
    # columns: offset bin. index: neuron. values: observed "P", or NaN if the neuron has no other neuron at that offset
    df_cc = df_cc.pivot(columns=["xz", "y"], index="node_id", values=use)
    # How to deal with NaN values? The default behavior of pandas is to ignore them, i.e. calculate correlation only
    # for observations where neither of a pair is NaN. 
    # A more conservative approach is to assume that the neuron would have the overall mean connection probability.
    # Finally we can add a small, random noise to all observations. That way, bins with all zeros will show up
    # to have zero correlation with all others. Else they will have NaN correlation
    if mode == "valid":
        ret = df_cc.corr()
    else:
        mn_P = df_in.groupby(["xz", "y"]).apply(lambda _x: (_x["N"] * _x[use]).sum() / _x["N"].sum())
        ret = (df_cc.fillna(mn_P) + 1E-9 * numpy.random.rand(*df_cc.shape)).corr()
    if thresh_pairs is not None:
        nncount = count_number_of_matching_pairs(df_cc)
        ret[nncount < thresh_pairs] = numpy.NaN
    return ret
    
CC_pre = observations_to_cc_of_offset_bins(df_n_p_pre, use="P")
CC_post = observations_to_cc_of_offset_bins(df_n_p_post, use="P")

In [ ]:
def translate_index(idx):
    idx = idx.to_frame().reset_index(drop=True)
    idx["xz"] = bin_centers_xz[idx["xz"]]
    idx["y"] = bin_centers_y[idx["y"]]
    return pandas.MultiIndex.from_frame(idx)

CC_post.index = translate_index(CC_post.index)
CC_post.columns = translate_index(CC_post.columns)
CC_pre.index = translate_index(CC_pre.index)
CC_pre.columns = translate_index(CC_pre.columns)

In [ ]:
out_fn = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/connectivity_higher_order_interactions_24-11-11.h5"

CC_pre.to_hdf(out_fn, key="{0}/{1}/pre".format(loaded, thresh))
CC_post.to_hdf(out_fn, key="{0}/{1}/post".format(loaded, thresh))

In [ ]:
mn_edge_count_pre = df_n_p_pre.groupby(["xz", "y"]).apply(lambda _x: _x["N"].mean())
mn_edge_count_pre.index = translate_index(mn_edge_count_pre.index)
mn_edge_count_post = df_n_p_post.groupby(["xz", "y"]).apply(lambda _x: _x["N"].mean())
mn_edge_count_post.index = translate_index(mn_edge_count_post.index)

In [ ]:
out_fn = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/connectivity_higher_order_interactions_24-11-11.h5"

mn_edge_count_pre.to_hdf(out_fn, key="{0}/{1}/pre_N".format(loaded, thresh))
mn_edge_count_post.to_hdf(out_fn, key="{0}/{1}/post_N".format(loaded, thresh))